In [1]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

from keras.models import Sequential
from keras.layers import Bidirectional, RNN, LSTM, GRU, Dense, BatchNormalization, Dropout
from keras.utils import print_summary, plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

Using TensorFlow backend.


In [2]:
# tensorflow config allow_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
KTF.set_session(sess)

In [3]:
DEFAULT_MAX_STR_LEN = 40
DEFAULT_W2V_DIM = 256

DEFAULT_RNN_ACTIVATION = 'tanh'
DEFAULT_RNN_UNIT = [256, 256]
DEFAULT_RNN_DROPOUT = [0.0, 0.0]
DEFAULT_RNN_R_DROPOUT = [0.0, 0.0]
DEFAULT_RNN_IN_SHAPE = [(DEFAULT_MAX_STR_LEN, DEFAULT_W2V_DIM)] + [ (DEFAULT_MAX_STR_LEN, u_size) for u_size in DEFAULT_RNN_UNIT[:-1]]
DEFAULT_RNN_RT_SEQ = [True]*(len(DEFAULT_RNN_UNIT)-1) + [False]

DEFAULT_DNN_ACTIVATION = 'selu'
DEFAULT_DNN_UNIT = [512, 512]
DEFAULT_DNN_BATCH_NORM = False
DEFAULT_DNN_DROPOUT = [0.5, 0.5]

rnn_model = Sequential()

# bi-directional rnn
for i in range(len(DEFAULT_RNN_UNIT)): 

    rnn_model.add(
        Bidirectional(
            GRU(
                input_shape=DEFAULT_RNN_IN_SHAPE[i],
                units=DEFAULT_RNN_UNIT[i], 
                activation=DEFAULT_RNN_ACTIVATION, recurrent_activation='hard_sigmoid', 
                use_bias=True, 
                kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', 
                dropout=DEFAULT_RNN_DROPOUT[i], recurrent_dropout=DEFAULT_RNN_R_DROPOUT[i], 
                return_sequences=DEFAULT_RNN_RT_SEQ[i], return_state=False
            ), 
            merge_mode='concat', weights=None
        )
    )

# dnn layers
for i in range(len(DEFAULT_DNN_UNIT)):
    
    rnn_model.add(
        Dense(
            units=DEFAULT_DNN_UNIT[i], activation=DEFAULT_DNN_ACTIVATION, use_bias=True,
            kernel_initializer='glorot_uniform', bias_initializer='zeros',
        )
    )
    if DEFAULT_DNN_BATCH_NORM: 
        rnn_model.add(BatchNormalization())
    rnn_model.add( Dropout(rate=DEFAULT_DNN_DROPOUT[i]) )


# output
rnn_model.add(
    Dense(
        units=2, activation='softmax', use_bias=True,
        kernel_initializer='glorot_uniform', bias_initializer='zeros',
    )
)